In [16]:
%matplotlib inline

import csv
import os
import json
import numpy as np
import pandas as pd
import itertools
from collections import defaultdict

In [2]:
EXCLUDE_PLAYERS = set([
    'debug2II1XV:debug8OX2HI',
    'A1AK9D1SN4YFF3:3R5F3LQFV2KC53TNCW558SS1SG1OZ0',
    'AIZTLQM7HHQN6:3JJVG1YBEBXUFVKP13XR0E2PNJK5BP',
    # 'A26LOVXF4QZZCO:324G5B4FB388ICVYRGFZEHK2NLL07G', # Maybe, since they are "quick" to jump to conclusion
])
HUMAN_DATAFILE = os.path.join('data', 'trialdata.csv')

def load_human_data():
    result = []
    # returns a map from participant ID to their results
    with open(HUMAN_DATAFILE, 'r') as f:
        reader = csv.reader(f)
        for line in reader:
            player_id = line[0]
            if player_id in EXCLUDE_PLAYERS:
                continue

            data = json.loads(line[3])
            if 'id' not in data or 'results' not in data:
                continue
            
            for stage, d in enumerate(data['results']):
                result.append({
                    'player': player_id,
                    'game': data['id'],
                    'stage': stage,
                    'p1': d[0] / 100,
                    'p2': d[1] / 100,
                    'p3': d[2] / 100
                })
            
    return pd.DataFrame(result, columns=['player', 'game', 'stage', 'p1', 'p2', 'p3'])

DATA = load_human_data()

In [3]:
# Normalize the data so we have both options

R_DATA = DATA.copy()
N_DATA = DATA.copy()
s = N_DATA.p1 + N_DATA.p2 + N_DATA.p3
N_DATA.p1 /= s
N_DATA.p2 /= s
N_DATA.p3 /= s

In [10]:
from proposal_game import load_games, ProposalGame, create_playthrough
from observer import observe_playthrough
from solver import Solver
GAMES = load_games()

VALID_GAMES = R_DATA.game.unique()

In [20]:
def run_analysis((k, gamma)):
    print (k, gamma)
    game = ProposalGame
    result = []
    solver = Solver(game, gamma=gamma)
    for game_id in VALID_GAMES:
        rounds = GAMES[game_id]
        playthrough = create_playthrough(rounds)
        observations = observe_playthrough(solver, playthrough, k)
        for stage, d in enumerate(observations):
            result.append({
                'k': k,
                'gamma': gamma,
                'game': game_id,
                'stage': stage,
                'p1': d[0],
                'p2': d[1],
                'p3': d[2]
            })
    return pd.DataFrame(result, columns=['k', 'gamma', 'game', 'stage', 'p1', 'p2', 'p3'])

In [ ]:
import multiprocessing
arguments = list(itertools.product(range(6), np.arange(0.5, 10.1, 0.5)))

pool = multiprocessing.Pool()
results = pool.map(run_analysis, arguments)
pool.close()
pool.join()

MODEL_PREDICTIONS = pd.concat(results)

(0, 4.5)
(1, 0.5)
(0, 6.5)
(0, 8.5)
(1, 4.5)
(1, 2.5)
(0, 2.5)
(0, 0.5)
(0, 7.0)
(0, 9.0)
(0, 3.0)
(0, 5.0)
(0, 1.0)
(0, 7.5)
(0, 9.5)
(0, 3.5)
(0, 5.5)
(0, 1.5)
(0, 4.0)
(0, 8.0)
(0, 10.0)
(0, 6.0)
(0, 2.0)
(1, 6.5)
(1, 8.5)
(2, 0.5)
(2, 4.5)
(2, 2.5)
(1, 3.0)
(1, 1.0)
(1, 5.0)
(1, 9.0)
(1, 7.0)
(1, 1.5)
(1, 5.5)
(1, 3.5)
(1, 9.5)
(1, 7.5)
(1, 4.0)
(1, 2.0)
(1, 6.0)
(1, 8.0)
(1, 10.0)
(2, 6.5)
(2, 8.5)
(3, 0.5)
(3, 2.5)
(3, 4.5)
(2, 1.0)
(2, 3.0)
(2, 5.0)
(2, 7.0)
(2, 9.0)
(2, 3.5)
(2, 5.5)
(2, 1.5)
(2, 7.5)
(2, 9.5)
(2, 4.0)
(2, 6.0)
(2, 2.0)
(3, 1.0)
(3, 3.0)
(3, 5.0)
(2, 8.0)
(2, 10.0)
(3, 6.5)
(3, 8.5)
(4, 0.5)
(4, 2.5)
(4, 4.5)
(3, 1.5)
(3, 3.5)
(3, 5.5)
(3, 7.0)
(3, 9.0)
(3, 2.0)
(3, 6.0)
(3, 4.0)
(4, 1.0)
(3, 7.5)
(3, 9.5)
(4, 3.0)
(4, 5.0)
(4, 6.5)
(4, 8.5)
(5, 0.5)
(3, 8.0)
(3, 10.0)
(4, 1.5)
(5, 2.5)
(5, 4.5)
(4, 3.5)
(4, 5.5)
(4, 7.0)
(4, 9.0)
(4, 2.0)
(5, 1.0)
(4, 6.0)
(4, 4.0)
(4, 9.5)
(4, 7.5)
(5, 3.0)
(5, 5.0)
(5, 6.5)
(5, 8.5)
(4, 10.0)
(4, 8.0)
(5, 1.5)
(5, 3.5)
(5, 5